<a href="https://colab.research.google.com/github/nvhieu0163/ComparativeOpinionMining_KLTN2023/blob/main/3_Framework_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tải thư viện

In [1]:
!pip install pytorch-crf

In [2]:
!pip install py_vncorenlp
!pip install transformers
!pip install tokenizers
!pip install pytorch-crf

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.0 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4307 sha256=34915cd7c8e13f6686a0f095efa768bc71c7d81f7880622cc43a19f9c4a0eed2
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load word sementator and tokenizer

In [4]:
import pandas as pd
import numpy as np
import py_vncorenlp
from transformers import AutoModel, AutoTokenizer

rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content/drive/MyDrive/NCKH_KLTN/Thuc_nghiem/vn_corenlp_2')
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchcrf import CRF

In [6]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
MAX_LENGTH_SENTENCE = 256

## PreProcessing

### Định nghĩa các hàm mapping words to bert token

In [23]:
def token_bert_mapping(tokenized_stc, seq_gold_token):
    seq_map, bert_index, token_index = {0: -1}, 1, 0
    seq_bert_token = tokenizer.convert_ids_to_tokens(tokenized_stc["input_ids"][0])

    print(seq_bert_token)
    print(seq_gold_token)
    while bert_index < len(seq_bert_token) and token_index < len(seq_gold_token):   # check chưa hết câu văn (không tính 'PAD')
        # print(index)
        seq_map[bert_index] = token_index  # khởi tạo


        # [UNK] denote special symbol
        if seq_bert_token[bert_index] == "<unk>":
            bert_index = bert_index + 1
            token_index = token_index + 1
            continue

        # get cur index correspond length
        token_length = len(seq_gold_token[token_index])     # độ dài của token
        bert_length = len(seq_bert_token[bert_index])       # độ dài của bert_token

        # drop "@@" prefix
        if seq_bert_token[bert_index].find("@@") != -1:     # tìm thấy subword
            bert_length = len(seq_bert_token[bert_index]) - 2   # sẽ không tính '@@', độ dài thực tế của bert_length phải trừ đi 2

        while token_length > bert_length:                       # chừng nào bert_length < token_length thì chưa đúng
            bert_index += 1
            seq_map[bert_index] = token_index
            bert_length += len(seq_bert_token[bert_index])
            if seq_bert_token[bert_index].find("@@") != -1:     # nếu vẫn còn subword, lặp lại quy trình
                bert_length -= 2

        assert bert_length == token_length, "appear mapping error! {}?{}\n{}\n{}".format(bert_length, token_length, seq_gold_token, seq_bert_token)

        token_index = token_index + 1       # cập nhật index
        bert_index = bert_index + 1         # cập nhật index

    # Do độ dài seq_bert_token lớn hơn seq_gold_token 2 (thêm '<s>' và '</s>')
    seq_map[bert_index] = -1    # token_index đã out of range, còn bert_index sẽ là '</s>'

    return seq_map

In [24]:
from collections import defaultdict

def mapping_output_to_actual_seq(segmented_stc_list, mapping_dict, y_pred):
    output = y_pred          # output hiện tại, kích thước 256
    assert len(output) == MAX_LENGTH_SENTENCE, {f'Error: {len(output)}'}

    # chuyển đổi dict từ bert2gold về gold2bert
    dict_bert2gold = mapping_dict
    dict_gold2bert = defaultdict(list)
    for key, value in dict_bert2gold.items():
        if value == -1:
            continue
        dict_gold2bert[value].append(key)

    # bỏ padding
    trancated_output = output[:len(dict_bert2gold)]

    pred = []
    for key in dict_gold2bert.keys():
        pred.append(trancated_output[dict_gold2bert[key][0]])

    assert len(pred) == len(segmented_stc_list), 'Đầu ra không bằng độ dài câu ban đầu'
    final_output = ''
    for i, token_label in enumerate(pred):
        if token_label == 1:
            final_output += segmented_stc_list[i] + ' '
    return final_output[:-1]

### Định nghĩa hàm Preprocessing

In [9]:
def preprocessing(stc):
    segmented_stc =  rdrsegmenter.word_segment(stc)
    assert len(segmented_stc) == 1
    segmented_stc_list = segmented_stc[0].split(' ')

    tokenized_stc = tokenizer(segmented_stc, padding='max_length', max_length = MAX_LENGTH_SENTENCE, truncation=True, return_tensors="pt")
    mapping_dict = token_bert_mapping(tokenized_stc, segmented_stc_list)

    return segmented_stc_list, tokenized_stc, mapping_dict

In [10]:
# print(preprocessing('Nhìn vào những thông số kích thước trên , VinFast Lux SA2.0 đang nhỉnh hơn Kia Sorento về mọi chỉ số , điều này giúp xe vạm vỡ hơn khi nhìn từ phần thân'))

## Định nghĩa pha 1

### Định nghĩa mô hình CSI

In [11]:
class CSI_model(nn.Module):
    def __init__(self):
        super(CSI_model, self).__init__()
        self.bert = AutoModel.from_pretrained('vinai/phobert-base')
        self.dropout1 = nn.Dropout(p = 0.2)
        self.linear1 = nn.Linear(self.bert.config.hidden_size, 384)
        self.dropout2 = nn.Dropout(p = 0.2)
        self.linear2 = nn.Linear(384, 192)
        self.dropout3 = nn.Dropout(p = 0.2)
        self.linear3 = nn.Linear(192, 96)
        self.dropout4 = nn.Dropout(p = 0.2)
        self.linear4 = nn.Linear(96, 3)
        # self.relu2 = nn.ReLU()
        # self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        last_hidden_state, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output1 = self.dropout1(pooled_output)
        linear_output_1 = self.linear1(dropout_output1)
        dropout_output2 = self.dropout2(linear_output_1)
        linear_output_2 = self.linear2(dropout_output2)
        dropout_output3 = self.dropout3(linear_output_2)
        linear_output_3 = self.linear3(dropout_output3)
        dropout_output4 = self.dropout4(linear_output_3)
        linear_output_4 = self.linear4(dropout_output4)
        # linear_output_2_relu = self.relu2(linear_output_2)
        return linear_output_4

### Định nghĩa hàm phase 1

In [12]:
def phase_1_processing(tokenized_stc, phase1_model):
    mask = tokenized_stc['attention_mask'].to(device)
    input_id = tokenized_stc['input_ids'].squeeze(1).to(device)

    phase1_model.eval()
    output = phase1_model(input_id, mask).to(device)

    _, phase_1_pred = torch.max(torch.softmax(output, dim=1), dim=1)
    return phase_1_pred

## Định nghĩa pha 2

### Định nghĩa mô hình joint CEE-SC

In [13]:
class CRFCell(nn.Module):
    def __init__(self, num_classes, batch_first=True):
        super(CRFCell, self).__init__()
        self.crf = CRF(num_tags=num_classes, batch_first=batch_first)

    def forward(self, feature, mask, target):
        mask = mask.bool()
        # feature : batch_size * seq_length * num_tags
        # mask : batch_size * seq_length
        # target : batch_size * seq_length

        # target is None means decode sequence
        if target is None:
            max_seq_len = feature.size(1)
            output = self.crf.decode(feature, mask)
            output = torch.as_tensor(
                [e + [0] * (max_seq_len - len(e)) for e in output],
                dtype=torch.long,
                device=feature.device
            )

        # target is not None means get loss
        else:
            output = self.crf(feature, target, mask, reduction='mean')
            output = output.neg()      # output: BATCH_SIZE
        return output

In [14]:
class CEE_SC_model(torch.nn.Module):
    def __init__(self, model_type, model_config):
        super(CEE_SC_model, self).__init__()

        self.model_type = model_type
        self.gamma_sc = model_config.get('gamma_sc')
        self.gamma_cee = model_config.get('gamma_cee')
        self.encoder = AutoModel.from_pretrained('vinai/phobert-base')
        self.embedding_dropout = nn.Dropout(p = model_config.get('dropout_threshold'))

        self.sentiment_linear_1 = nn.Linear(self.encoder.config.hidden_size, 96)
        self.sentiment_linear_2 = nn.Linear(96, 3)                              # better, equal, worse

        self.subject_linear =  nn.Linear(self.encoder.config.hidden_size, 2)
        self.object_linear =  nn.Linear(self.encoder.config.hidden_size, 2)
        self.aspect_linear = nn.Linear(self.encoder.config.hidden_size, 2)

        self.crf_subject_layer = CRFCell(2, batch_first = True)
        self.crf_object_layer = CRFCell(2, batch_first = True)
        self.crf_aspect_layer = CRFCell(2, batch_first = True)


    def forward(self, input_ids, attn_mask, sentiment_label=None, sub=None, obj=None, aspect=None):
        # get hidden representation (token embedding)
        token_embedding, pooled_output = self.encoder(input_ids, attn_mask, return_dict=False) # token_embedding = last_hidden_state
        batch_size, sequence_length, hidden_size = token_embedding.size()

        # drop out
        final_embedding = self.embedding_dropout(token_embedding)       # batch_size * sequence_length * hidden_size
        # class_embedding = self.embedding_dropout(pooled_output)

        # linear mapping.
        subject_linear_output = self.subject_linear(final_embedding)    # batch_size * sequence_length * num_label
        object_linear_output = self.object_linear(final_embedding)      # tức là bây h mỗi token chỉ được biểu diễn bởi vector 3 giá trị
        aspect_linear_output = self.aspect_linear(final_embedding)

        # get SC
        sentiment_avg_output = torch.mean(final_embedding, dim = 1)     # batch_size * hidden_size
        sentiment_linear_1_output = self.sentiment_linear_1(sentiment_avg_output)    # batch_size * 96
        sentiment_class_prob = self.sentiment_linear_2(sentiment_linear_1_output)     # batch_size * 3
        # sent_class_prob = self.sent_linear(class_embedding)

        # TESTING: Chỉ lấy đầu ra
        if sub is None and sentiment_label is None and obj is None and aspect is None:
            sentiment_class_output = sentiment_class_prob                       # batch_size * num_label

            subject_crf_output = self.crf_subject_layer(subject_linear_output, attn_mask, None)     # batch_size * sequence_length
            object_crf_output = self.crf_object_layer(object_linear_output, attn_mask, None)
            aspect_crf_output = self.crf_aspect_layer(aspect_linear_output, attn_mask, None)

            return sentiment_class_output, subject_crf_output, object_crf_output, aspect_crf_output

        # TRAINING: Lấy loss
        sentiment_loss = F.cross_entropy(sentiment_class_prob, sentiment_label.view(-1).to(device))

        sub_loss = self.crf_subject_layer(subject_linear_output, attn_mask, sub.to(device))
        obj_loss = self.crf_object_layer(object_linear_output, attn_mask, obj.to(device))
        asp_loss = self.crf_aspect_layer(aspect_linear_output, attn_mask, aspect.to(device))
        element_loss = sub_loss + obj_loss + asp_loss

        # according different model type to get different loss type.
        if self.model_type == "classification":
            return sentiment_loss
        elif self.model_type == "extraction":
            return element_loss
        else:
            assert self.model_type == 'joint'
            return self.gamma_sc * sentiment_loss + self.gamma_cee * element_loss

### Định nghĩa hàm phase 2

In [15]:
def phase_2_processing(segmented_stc, processed_stc, mapping_dict, phase_2_model):
    mask = processed_stc['attention_mask'].to(device)
    input_id = processed_stc['input_ids'].squeeze(1).to(device)

    phase_2_model.eval()
    sent_output_pred, sub_pred, obj_pred, aspect_pred = phase_2_model(input_id, mask)

    # print(sent_output_pred.shape)
    # print(sub_pred.shape)

    _, sentiment_pred = torch.max(torch.softmax(sent_output_pred, dim=1), dim=1)
    sub = mapping_output_to_actual_seq(segmented_stc, mapping_dict, sub_pred[0])
    obj = mapping_output_to_actual_seq(segmented_stc, mapping_dict, obj_pred[0])
    aspect = mapping_output_to_actual_seq(segmented_stc, mapping_dict, aspect_pred[0])

    sentiment_dict = {0: 'Worse', 1 : 'Equal', 2 : 'Better'}
    return {'Subject' : sub , 'Object' : obj, 'Aspect' : aspect, 'Sentiment' : sentiment_dict[sentiment_pred.item()]}

## Main

In [27]:
def main(stc, phase1_model, phase2_model):
    # Data Preprocessing
    print("========= Pre Processing =========")
    segmented_stc_list, tokenized_stc, mapping_dict = preprocessing(stc)

    print("\n========= Compapative Opinion Mining =========")
    # Phase 1 Processing
    phaseI_output = phase_1_processing(tokenized_stc, phase1_model).item()
    # print('Nhãn pha 1: ', phaseI_output)

    # Phase 2 Processing
    if phaseI_output == 0:
        return "Prediction: Non Comparative Sentence."
    elif phaseI_output == 1:
        return "Prediction: Non-gradable Comparative Sentence."
    else:
        phaseII_output = phase_2_processing(segmented_stc_list, tokenized_stc, mapping_dict, phase2_model)
        return f"Prediction: Gradable Comparative Sentence:\n{phaseII_output}"

### Load mô hình

In [ ]:
phase1_model = CSI_model()
phase1_model.load_state_dict(torch.load('/content/drive/MyDrive/phase2_saved_model/csi_model.pth'))
phase1_model.cuda()
if torch.cuda.device_count() > 1:
    phase1_model = nn.DataParallel(phase1_model)
    phase1_model = phase1_model.module

In [ ]:
model_config = {
    'gamma_sc' : 1,
    'gamma_cee' : 1,
    'dropout_threshold' : 0
}

phase2_model = CEE_SC_model('joint', model_config)
phase2_model.load_state_dict(torch.load('/content/drive/MyDrive/phase2_saved_model/joint_model.pth'))
phase2_model.cuda()
if torch.cuda.device_count() > 1:
    phase2_model = nn.DataParallel(phase2_model)
    phase2_model = phase2_model.module

### Demo

In [19]:
example_stc = "Honda CR-V 1.5L có cùng giá bán với V-Series 2.5 SV Luxury, ở mức 1,083 tỷ đồng"

In [28]:
output = main(example_stc, phase1_model, phase2_model)
print(output)

========= Pre Processing =========
['<s>', 'Honda', 'CR-V', '1.5', 'L', 'có', 'cùng', 'giá', 'bán', 'với', 'V-@@', 'Series', '2.5', 'SV', 'Luxury', ',', 'ở', 'mức', '1,0@@', '83', 'tỷ', 'đồng', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>

In [37]:
example_stc = "Toyota Camry có động cơ 2.8L"
output = main(example_stc, phase1_model, phase2_model)
print(output)

========= Pre Processing =========
['<s>', 'Toyota', 'Camry', 'có', 'động_cơ', '2.8', 'L', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '

In [38]:
example_stc = "Toyota Camry có động cơ mạnh hơn Xpander"
output = main(example_stc, phase1_model, phase2_model)
print(output)

========= Pre Processing =========
['<s>', 'Toyota', 'Camry', 'có', 'động_cơ', 'mạnh', 'hơn', 'X@@', 'p@@', 'ander', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<

In [39]:
example_stc = "Toyota Camry có động cơ khác với Xpander"
output = main(example_stc, phase1_model, phase2_model)
print(output)

========= Pre Processing =========
['<s>', 'Toyota', 'Camry', 'có', 'động_cơ', 'khác', 'với', 'X@@', 'p@@', 'ander', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<